# Índice

1. [Preparacion del Dataset](#preparaciondataset)

    1.1 [Visualización de los datos](#visualizaciondatos) <br>
    1.2 [Funciones Útiles](#fu) <br>
    1.3 [Carga del Dataset](#cargadeldataset) <br>
    1.4 [Modificación de imagenes a baja y media resolución](#modimgs) <br>
    1.5 [Creación de los datasets](#creaciondatasets)
    
    - [a). Baja resolución (256x256)](#br) <br>
    - [b). Media resolución (512x512)](#mr)<br>
    - [c). Alta resolución (1024x1024)](#hr)
2. [Creación y Entrenamiento de modelos](#creacionentrenamientomodelos)

    2.1 [Autoencoder (Vanilla)](#ae) <br>
    &emsp;[Introducción](#aei) <br>
    &emsp;2.1.1 [Creación del Modelo](#aecm) <br>
    &emsp;2.1.2 [Entrenamiento del Modelo](#aeem) <br>
    &emsp;2.1.3 [Resultados del entrenamiento](#aer) <br>

    2.2 [Variational Auto Encoder (VAE)](#vae) <br>
    &emsp;[Introducción](#vaei) <br>
    &emsp;2.2.1 [Creación del Modelo V1](#vaecm1) <br>
    &emsp;2.2.2 [Entrenamiento del modelo V1](#vaeemv1) <br>
    &emsp;2.2.3 [Resultados del entrenamiento V1](#vaerv1) <br>
    &emsp;2.2.4 [Creación del Modelo V2](#vaecmv2) <br>
    &emsp;2.2.5 [Entrenamiento del Modelo V2](#vaeemv2) <br>
    &emsp;2.2.6 [Resultados del entrenamiento V2](#vaermv2) <br>
    
    2.3 [GAN](#gan) <br>
    &emsp;[Introducción a las redes GAN](#gani) <br>
    &emsp;2.3.1 [Declaración de los Generadores](#dgan) <br>
    &emsp;2.3.2 [Declaración de las funciones de pérdida](#dfp) <br>
    &emsp;2.3.3 [Declaración del discriminador](#dd) <br>
    &emsp;2.3.4 [Declaracion de las funciones de Entrenamiento](#dfe) <br>
    &emsp;2.3.5 [Entrenamiento y comprobación del modelo](#ganecm) <br>
    
    2.4 [GAN con U-Net o pix2pix](#cgan) <br>
    &emsp;2.4.1 [Declaración de los Generadores](#dunet) <br>
    &emsp;2.4.2 [Declaración de los discriminadores y entrenamiento](#unetde) <br>
    
    2.5 [EDSRGAN](#edsr) <br>
    &emsp;2.5.1 [Creación del generador con arquitectura EDSR](#edsr) <br>
    &emsp;2.5.2 [Declaración de los discriminadores y entrenamiento](#edsrde)<br>
3. [Comparación de modelos](#comparacionmodelos)
    
    3.1 [Carga de modelos](#cm)<br>
    3.2 [Comparativa de imágenes](#ci)<br>
    3.3 [Tablas comparativas de métricas](#tcm) <br>


Imports:

In [ ]:
import tensorflow as tf
import numpy as np
import os
from PIL import Image, ImageFilter
import matplotlib.pyplot as plt
import cv2
import math
import time
from IPython import display
import pandas as pd
from tensorflow.keras import backend as bk

# 1. Preparación del Dataset <a id="preparaciondataset"></a>

## 1.1 Visualización de los datos <a id="visualizaciondatos"></a>

Se va a abrir un par de imágenes para ver el contenido del dataset.

In [ ]:
show_image = Image.open("./Dataset/DIV2K_train_HR/0001.png")
show_image.size

In [ ]:
plt.imshow(show_image)

In [ ]:
show_image = Image.open("./Dataset/DIV2K_train_HR/0124.png")
show_image.size

In [ ]:
plt.imshow(show_image)

Como se puede observar, las imágenes tienen un tamaño constante de 2040 de ancho o de alto.

## 1.2 Funciones útiles: <a id="fu"></a>

Esta función enseña las diferencias entre la imagen de input, la imagen real y la predicha:

In [ ]:
def plot_model_diff(model,valid_dataset=None, input=None, target=None,figsize=(15,15),title=""):

    if valid_dataset is not None:
        input, target = next(iter(valid_dataset))
        input = input.numpy()
        target = target.numpy()

    plt.figure(figsize=figsize)

    plt.subplot(1, 3, 1)
    plt.imshow(input[0]) 
    plt.title('Input')

    plt.subplot(1, 3, 2)
    plt.imshow(target[0])  
    plt.title('Ground Truth')
    
    s = model.predict(input,verbose=0)[0]

    plt.subplot(1, 3, 3)
    plt.imshow(s)
    plt.title('Prediction')

    plt.suptitle(title)
    plt.subplots_adjust(top=1.5)

    plt.show()

Esta funcion muestra un ejemplo del dataset:

In [ ]:
def plot_dataset(valid_dataset=None,input=None,target=None, dim=2):

    if valid_dataset is not None:
        if dim == 2:
            input, target = next(iter(valid_dataset))
            input = input.numpy()
            target = target.numpy()
        elif dim == 1:
            input = next(iter(valid_dataset))
            input = input.numpy()
            target = input
        else:
            print("Invalid dim when passing a dataset")

    plt.figure(figsize=(15, 15))

    plt.subplot(1, 2, 1)
    plt.imshow(input[0]) 
    plt.title('Input')

    plt.subplot(1, 2, 2)
    plt.imshow(target[0])  
    plt.title('Ground Truth')

    plt.show()

## 1.3 Carga del Dataset <a id="cargadeldataset"></a>

Se definen algunas variables globales como las rutas de los directorios del dataset

In [ ]:
DIV2K_TRAIN_HR_PATH = "Dataset/DIV2K_train_HR/*.png"
DIV2K_VALID_HR_PATH = "Dataset/DIV2K_valid_HR/*.png"

En este caso se está haciendo uso del dataset DIV2k

## 1.4 Modificación de imagenes a baja y media resolución <a id="modimgs"></a>

 Esta función añade desenfoque gaussiano dependiendo del tamaño del kernel y del valor sigma:
 (Nota: obtenido de: https://gist.github.com/blzq/c87d42f45a8c5a53f5b393e27b1f5319)

In [ ]:
def gaussian_blur(img, kernel_size=2, sigma=50):
    def gauss_kernel(channels, kernel_size, sigma):
        ax = tf.range(-kernel_size // 2 + 1.0, kernel_size // 2 + 1.0)
        xx, yy = tf.meshgrid(ax, ax)
        kernel = tf.exp(-(xx ** 2 + yy ** 2) / (2.0 * sigma ** 2))
        kernel = kernel / tf.reduce_sum(kernel)
        kernel = tf.tile(kernel[..., tf.newaxis], [1, 1, channels])
        return kernel

    gaussian_kernel = gauss_kernel(tf.shape(img)[-1], kernel_size, sigma)
    gaussian_kernel = gaussian_kernel[..., tf.newaxis]

    img = tf.expand_dims(img, axis=0)  
    img_blurred = tf.nn.depthwise_conv2d(img, gaussian_kernel, [1, 1, 1, 1],
                                          padding='SAME', data_format='NHWC')
    img_blurred = tf.squeeze(img_blurred, axis=0)  

    return img_blurred

Se añade una máscara y se define la cantidad de ruido y lo grande que es la máscara

In [ ]:
def corrupt_part_of_image(image, noise_level, corruption_level=0.04):
    
    mask = tf.random.uniform(tf.shape(image)[:2]) < corruption_level
    noise = tf.random.normal(tf.shape(image), stddev=noise_level)
    noise = (noise + 1.0) / 2.0
    corrupted_image = tf.where(mask[..., tf.newaxis], tf.clip_by_value(image + noise, 0.0, 1.0), image)
    
    return corrupted_image

Función que añade ruido y desenfoque

In [ ]:
def random_jitter(img,noise=0.1,corrupt=0.04,blur=2):
    img = gaussian_blur(img,kernel_size=blur)
    img = corrupt_part_of_image(img,noise_level=noise,corruption_level=corrupt)
    return img

Función para cargar una imagen png y transformarla a float32

In [ ]:
def load(input_path):
    img = tf.io.read_file(input_path)
    img = tf.cast(tf.image.decode_png(img,channels=3),tf.float32)
    return img

Función para normalizar la imagen

In [ ]:
def normalize(img):
    #img = (img / 127.5) - 1 # Normalización de la imagen entre [-1 y 1]
    img = img / 255 #Normalización de la imagen entre [0 y 1]
    return img

Funciones para crear una canalización de datos y procesar las imágenes de alta resolución:

In [ ]:
#Target = Ground Truth ; Input = Imagen que se va a meter al modelo

# Salida: 256 y 1024
def low_res_img_map(input_path):
    target_img = load(input_path)
    target_img = tf.image.resize(target_img,size=(1024,1024))
    
    input_image = tf.image.resize(target_img,size=(256,256))

    input_image = random_jitter(input_image)

    target_img = normalize(target_img)
    input_image = normalize(input_image)

    return input_image, target_img

# Salida: 512 y 1024
def med_res_img_map(input_path):
    target_img = load(input_path)
    target_img = tf.image.resize(target_img,size=(1024,1024))
    
    input_image = tf.image.resize(target_img,size=(512,512))

    input_image = random_jitter(input_image,corrupt=0.1)

    target_img = normalize(target_img)
    input_image = normalize(input_image)

    return input_image, target_img

# Saalda: 1024 y 1024
def high_res_img_map(input_path):
    target_img = load(input_path)
    target_img = tf.image.resize(target_img,size=(1024,1024))

    input_image = random_jitter(target_img,noise=0.4,corrupt=0.2,blur=8)

    target_img = normalize(target_img)
    input_image = normalize(input_image)

    return input_image, target_img

## 1.5 Creación de los datasets <a id="creaciondatasets"></a>

Se va a crear una canalización para que las imágenes se procesen en la cpu en tiempo de entrenamiento y no en GPU:

In [ ]:
def create_dataset(dataset_path, res_func, batch_size=1):
    dataset = tf.data.Dataset.list_files(str(dataset_path))
    dataset = dataset.map(res_func, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    return dataset

### a). Baja resolución (256x256) <a id="br"></a>

In [ ]:
low_res_train_dataset = create_dataset(DIV2K_TRAIN_HR_PATH,low_res_img_map)
low_res_valid_dataset = create_dataset(DIV2K_VALID_HR_PATH,low_res_img_map)

In [ ]:
plot_dataset(low_res_train_dataset)

### b). Media resolución (512x512) <a id="mr"></a>

In [ ]:
med_res_train_dataset = create_dataset(DIV2K_TRAIN_HR_PATH,med_res_img_map)
med_res_valid_dataset = create_dataset(DIV2K_VALID_HR_PATH,med_res_img_map)

In [ ]:
plot_dataset(med_res_train_dataset)

### c). Alta resolución (1024x1024) <a id="hr"></a>

In [ ]:
high_res_train_dataset = create_dataset(DIV2K_TRAIN_HR_PATH,high_res_img_map)
high_res_valid_dataset = create_dataset(DIV2K_VALID_HR_PATH,high_res_img_map)

In [ ]:
plot_dataset(high_res_train_dataset)

# 2. Creación y Entrenamiento de modelos <a id="creacionentrenamientomodelos"></a>

Se comprueba que se detecta la tarjeta gráfica:

In [ ]:
tf.config.list_physical_devices()

## 2.1 Autoencoder (Vanilla) <a id="ae"></a>

### Introducción <a id="aei"></a>

Un autoencoder es un tipo de red de neuronas capaz de aprender características de los datos, reducir su dimensionalidad y reconstruir estos mismos manteniendo parte de su fidelidad. Es decir, consta de tres partes: *codificador*, *espacio latente* y *decodificador*.

El *codificador* o en ingles *encoder* se puede representar de la siguiente forma:

$h_i = g(X_i)$ , donde $h_i ∈ R^q$ que simboliza el *espacio latente*;

El *decodificador* o en ingles *decoder* se puede representar de la siguiente forma:

$\tilde{x}_i = f(h_i) = f(g(x_i))$ , donde $\tilde{x}_i ∈ R^n$ ;

Para el entrenamiento del autoencoder, hay que buscar las funciones f y g que minimicen la diferencia entre $x_i$ y $\tilde{x}_i$:

$argmin_{f,g} <[∆(x_i,f(g(x_i)))]>$ ;

$<·>$, Indica media de todas las desviaciones observadas.

![Auto Encoder Vanilla](./Images/latent_representation_AE.png)

### 2.1.1 Creación del Modelo <a id="aecm"></a>

Se va a crear una clase autoencoder un tanto diferente, para este autoencoder se van ha utilizar entradas de datos diferentes a la salida. Se va ha intentar reescalar las imágenes de 256x256 y 512x512 a 1024x1024, añadiendo 2 y 1 capas más de convolución respectivamente:

In [ ]:
class Autoencoder(tf.keras.models.Model):
    def __init__(self,input_shape):
        super().__init__()
        
        #Codificador
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.Input(shape=input_shape),
            tf.keras.layers.Convolution2D(128, (3, 3), activation='relu', padding='same'),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Convolution2D(256, (3, 3), activation='relu', padding='same'),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Convolution2D(512, (3, 3), activation='relu', padding='same'),
            tf.keras.layers.MaxPooling2D((2, 2)),
        ])

        #Decodificador
        self.decoder = tf.keras.Sequential([
            tf.keras.layers.Convolution2D(512, (3, 3), activation='relu', padding='same'),
            tf.keras.layers.UpSampling2D((2, 2)),
            tf.keras.layers.Convolution2D(256, (3, 3), activation='relu', padding='same'),
            tf.keras.layers.UpSampling2D((2, 2)),
            tf.keras.layers.Convolution2D(128, (3, 3), activation='relu', padding='same'),
            tf.keras.layers.UpSampling2D((2, 2)),
        ])

        #Si la imagen tiene un tamaño de 1024 la salida son 3 filtros (RGB)
        if input_shape[0] == 1024:
            self.decoder.add(tf.keras.layers.Convolution2D(3, (3, 3), activation='sigmoid', padding='same'))
        
        #Si la imagen tiene un tamaño de 512 la salida es una capa convolucional mas 3 filtros (RGB)
        if input_shape[0] == 512:
            self.decoder.add(tf.keras.layers.Convolution2D(64, (3, 3), activation='relu', padding='same'))
            self.decoder.add(tf.keras.layers.UpSampling2D((2, 2)))
            self.decoder.add(tf.keras.layers.Convolution2D(3, (3, 3), activation='sigmoid', padding='same'))

        #Si la imagen tiene un tamaño de 256 la salida son dos capas convolucionales mas 3 filtros (RGB)
        if input_shape[0] == 256:
            self.decoder.add(tf.keras.layers.Convolution2D(64, (3, 3), activation='relu', padding='same'))
            self.decoder.add(tf.keras.layers.UpSampling2D((2, 2)))
            self.decoder.add(tf.keras.layers.Convolution2D(32, (3, 3), activation='relu', padding='same'))
            self.decoder.add(tf.keras.layers.UpSampling2D((2, 2)))
            self.decoder.add(tf.keras.layers.Convolution2D(3, (3, 3), activation='sigmoid', padding='same'))


    def call(self, x):
        return self.decoder(self.encoder(x))

### 2.1.2 Entrenamiento del modelo <a id="aeem"></a>

Se declaran los tres modelos: 256,512 y 1024:

In [ ]:
autoencoder_256 = Autoencoder(input_shape=(256,256,3))
autoencoder_512 = Autoencoder(input_shape=(512,512,3))
autoencoder_1024 = Autoencoder(input_shape=(1024,1024,3))

Se entrena cada red con pérdida MSE y con el optimizador ADAM:

Se mira la pérdida del modelo:

In [ ]:
autoencoder_1024.compile(loss='mean_squared_error', optimizer='adam')
history_high_res = autoencoder_1024.fit(high_res_train_dataset, epochs=25,validation_data=high_res_valid_dataset)

In [ ]:
pd.DataFrame(history_high_res.history).plot()
plt.xlabel('Epoch num.')
plt.show()

In [ ]:
autoencoder_512.compile(loss='mean_squared_error', optimizer='adam')
history_mid_res = autoencoder_512.fit(med_res_train_dataset, epochs=30,validation_data=med_res_valid_dataset)

In [ ]:
pd.DataFrame(history_mid_res.history).plot()
plt.xlabel('Epoch num.')
plt.show()

In [ ]:
autoencoder_256.compile(loss='mean_squared_error', optimizer='adam')
history_low_res = autoencoder_256.fit(low_res_train_dataset, epochs=40,validation_data=low_res_valid_dataset)

In [ ]:
pd.DataFrame(history_low_res.history).plot()
plt.xlabel('Epoch num.')
plt.show()

Se guardan los modelos:

In [ ]:
autoencoder_256.save("Checkpoints/Autoencoder/autoencoder_256_model")
autoencoder_512.save("Checkpoints/Autoencoder/autoencoder_512_model")
autoencoder_1024.save("Checkpoints/Autoencoder/autoencoder_1024_model")

### 2.1.3 Resultados del entrenamiento: <a id="aer"></a>

256->1024

In [ ]:
plot_model_diff(autoencoder_256,low_res_valid_dataset)

512->1024

In [ ]:
plot_model_diff(autoencoder_512,med_res_valid_dataset)

1024->1024

In [ ]:
plot_model_diff(autoencoder_1024,high_res_valid_dataset)

Como podemos observar los resultados no son para nada malos, pero pueden mejorarse:

## 2.2 Variational Auto Encoder (VAE) <a id="vae"></a>

### Introducción: <a id="vaei"></a>

Teniendo nuestro autoencoder ya creado, lo que se busca con un variational autoencoder es, con la misma estructura, intentar regularizar mejor los datos como se ve en el gráfico: <br><br>
![vae](./Images/variational_autoencoder.png)

Siendo $x∈X^D$ un vector de variables observables, donde $X\sube R$ o $X\sube Z$ donde $Z\in R^M$ sea un vector de variables latentes. Las variables latentes son variables que están dentro del modelo, osea que no forman parte del dataset. Para representar en forma de red Bayesiana o modelos gráficos dirigidos se necesita la probabilidad conjunta de x y z: $p_\theta(x,z)dz$. 

Para sacar la similitud marginal o la evidencia con las propiedades de la probabilidad conjunta para datos continups se tiene: 

$p_\theta(x)=\int {p_\theta(x,z)dz}$. 

Para entrenar el modelo es necesario despejar z cosa que es computacionalmente imposible tratar este tipo de problema así que se opta por otra solución y es intentar maximizar la función ELBO (evidence lower bound):  

(ELBO) => $\ln{p_\theta(x)} \le E_{z\text{\textasciitilde}q_\phi(z/x)}[\ln{p_\theta(x/z)} + \ln{p_\theta(z)} - \ln{q_\phi(z/x)}]$

- Donde $q_\phi(z/x)=N(\mu_\phi(x),\sigma_\phi^2(x)I)$ es el *codificador*
- Donde $p_\theta(x/z)=N(\mu,\varSigma)$ es el *decodificador*
- Donde $p_\lambda(z)=N(0,1)$ es el *prior* o la *similitud marginal*

![vae_model](./Images/variational_autoencoder_model.png)

Otra forma mucho más útil de escribir el ELBO es agrupar la parte del *prior* y del *decodificador* convirtiéndolo en una divergencia *KL(Kullback-Leibler)*

- $\ln{p_\theta(x)} \le E_{z\text{\textasciitilde}q_\phi(z/x)}[\ln{p_\theta(x/z)} + \textcolor{#FF33F3}{\ln{p_\theta(z)}} - \textcolor{#FF33F3}{\ln{q_\phi(z/x)}}] =$
- $ = -\textcolor{#FF33F3}{KL(q_\phi(z/x)||p(z))} + E_{z\text{\textasciitilde}q_\phi(z/x)}[\ln{p_\theta(x/z)}]$

La divergencia *KL(Kullback-Leibler)* es una medida de la diferencia entre dos distribuciones de probabilidad y se escribe así: 

- $D_{kl}(P||Q)=\sum_xP(x)log \frac {P(x)} {Q(x)}$ para valores continuos y discretos.

Esto es útil cuando el valor de KL es fácil de analizar siendo: 

- $-\textcolor{#FF33F3}{KL(q_\phi(z/x)||p(z))}= \textcolor{#DAF7A6}{\frac {1}{2}(1 + \ln\sigma_\phi^2(x) - \sigma_\phi^2(x) - \mu_\phi(x)^2)}$  

Siendo este valor el valor de pérdida del modelo al que se le va ha añadir el error l2 con un factor de balance B para que sea mas fiel la salida. Entonces, el error del modelo será: 

- $loss = B*l2(x,(x)´) + (\textcolor{#DAF7A6}{\frac {1}{2}(1 + \ln\sigma_\phi^2(x) - \sigma_\phi^2(x) - \mu_\phi(x)^2)})$

El modelo, además, se va ha entrenar con *SGD* o *descenso del gradiente estocástico*, con lo que se llama *reparametrization trick*, que busca poder diferenciar las varaibles aleatorias continuas respecto de sus parámetros. En el caso del VAE, tenemos un vector de medias $\mu_\theta(x)$ y el vector de desviaciones típicas $\sigma_\theta^2(x)$. En lugar de muestrear directamente esta distribución gaussiana $(\mu,\sigma)$, se puede muestrear una distribución gaussiana estándar  $N(0,1)$, y luego transformar este muestreo mediante la siguiente fórmula:

- $z = \mu + \sigma \bigodot \epsilon$ ; Donde $\bigodot$ denota la multiplicación de cada elemento por $\epsilon$, que es una distribución gaussiana estandar (ruido).

### 2.2.1 Creación del modelo V1 <a id="vaecm1"></a>

En una primera aproximación se va a crear un VAE con convolucionales que coja imágenes de 1024x1024x3 y las intente recrear. Para ello, se va a crear una canalización de datos que coja las imágenes del dataset y las transforme a 1024x1024: 

In [ ]:
def vae_map(input_path):
    
    target_img = load(input_path)
    target_img = tf.image.resize(target_img,size=(1024,1024))
    target_img = normalize(target_img)

    return target_img

In [ ]:
vae_train_dataset = create_dataset(DIV2K_TRAIN_HR_PATH,vae_map,batch_size=2)
vae_valid_dataset = create_dataset(DIV2K_VALID_HR_PATH,vae_map)

En cuanto a la creación del modelo, se utilizan capas convolucionales que disminuyen el tamaño de la imagen para poder evitar errores de memoria (OOM):

In [ ]:
def CVAE(latent_dim):

  #Encoder
  encoder_input = tf.keras.layers.Input(shape=(1024,1024,3))
  
  x = tf.keras.layers.Conv2D(filters=16,  kernel_size=3, activation='relu', padding='same')(encoder_input) # 512x512x16
  x = tf.keras.layers.Conv2D(filters=16,  kernel_size=3, strides=2, activation='relu', padding='same')(x) # 512x512x16
  x = tf.keras.layers.Conv2D(filters=32,  kernel_size=3, strides=2, activation='relu', padding='same')(x) # 256x256x32
  x = tf.keras.layers.Conv2D(filters=64,  kernel_size=3, strides=2, activation='relu', padding='same')(x) # 128x128x64
  x = tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=2, activation='relu', padding='same')(x) # 64x64x128
  x = tf.keras.layers.Conv2D(filters=256, kernel_size=3, strides=2 ,activation='relu', padding='same')(x) # 32x32x256

  s = bk.int_shape(x)
  x = tf.keras.layers.Flatten()(x)
  
  z_mean = tf.keras.layers.Dense(latent_dim)(x)
  z_log_var = tf.keras.layers.Dense(latent_dim)(x)

  @tf.function
  def sampling(args):
      z_mean, z_log_var = args
      epsilon = bk.random_normal(shape=(bk.shape(z_mean)[0], latent_dim)) # Vec
      return z_mean + bk.exp(z_log_var / 2) * epsilon

  # Reparameterization trick
  z = tf.keras.layers.Lambda(sampling)([z_mean, z_log_var])
  
  encoder = tf.keras.Model(encoder_input,[z_mean,z_log_var,z], name="encoder")

  #Decoder
  decoder_input = tf.keras.layers.Input(shape=bk.int_shape(z)[1:])
  x = tf.keras.layers.Dense(np.prod(s[1:]), activation='relu')(decoder_input)
  x = tf.keras.layers.Reshape(s[1:])(x)
  
  x = tf.keras.layers.Conv2DTranspose(filters=256, kernel_size=3, strides=2, padding='same', activation='relu')(x) # 64x64x256
  x = tf.keras.layers.Conv2DTranspose(filters=128, kernel_size=3, strides=2, padding='same', activation='relu')(x) # 128x128x128
  x = tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=3,  strides=2, padding='same', activation='relu')(x) # 256x256x64
  x = tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=3,  strides=2,padding='same', activation='relu')(x) # 512x512x32
  x = tf.keras.layers.Conv2DTranspose(filters=16, kernel_size=3,  strides=2,padding='same', activation='relu')(x) # 1024x1024x16
  x = tf.keras.layers.Conv2DTranspose(filters=16, kernel_size=3 ,padding='same', activation='relu')(x) # 1024x1024x16
  x = tf.keras.layers.Conv2DTranspose(filters=3, kernel_size=3, strides=1, activation='sigmoid', padding='same')(x) # 1024x1024x3

  decoder = tf.keras.models.Model(decoder_input,x, name="decoder")
  decoder_output = decoder(encoder(encoder_input)[2])

  #Creación del modelo
  vae = tf.keras.models.Model(encoder_input, decoder_output, name = "vae")

  #Valores de pérdida
  reconstruction_loss = tf.keras.losses.mse(bk.flatten(encoder_input), bk.flatten(decoder_output))
  reconstruction_loss *= 1024 * 1024 * 3
  kl_loss = -0.5 * bk.sum(1 + z_log_var - bk.square(z_mean) - bk.exp(z_log_var), axis=1)
  B = 1000   
  vae_loss = bk.mean(B * reconstruction_loss + kl_loss)
  vae.add_loss(vae_loss)
  vae.add_metric(kl_loss, name="kl_loss")
  vae.add_metric(reconstruction_loss, name="reconstruction_loss")
  vae.compile(optimizer='adam')

  return vae

Se construye el modelo con un espacio latente de 256:

In [ ]:
cvae = CVAE(256)

Se comprueba la estructura del modelo:

In [ ]:
tf.keras.utils.plot_model(cvae, show_shapes=True, dpi=64)

### 2.2.2 Entrenamiento del modelo V1 <a id="vaeemv1"></a>

Se entrena el modelo con 500 epochs:

In [ ]:
cvae_history = cvae.fit(vae_train_dataset,shuffle=True,epochs=500)

Se guarda el modelo:

In [ ]:
cvae.save("Checkpoints/VAE/VAE_1024_model")

### 2.2.3 Resultados de entrenamiento <a id="vaerv1"></a>

Se crea una función para mostrar las diferencias entre la entrada y la salida:

In [ ]:
def plot_model_diff_vae(model,valid_dataset):

    input  = next(iter(valid_dataset))

    input = input.numpy()

    plt.figure(figsize=(15, 15))

    plt.subplot(1, 2, 1)
    plt.imshow(input[0]) 
    plt.title('Input')

    plt.subplot(1, 2, 2)

    plt.imshow(model.predict(input, verbose=0)[0]) 
    plt.title('Prediction')

    plt.show()

Se mustran las imágenes, una del conjunto de validación y la otra del conjunto de entrenamiento:

In [ ]:
plot_model_diff_vae(cvae,vae_valid_dataset)

Se va a ver como predice una imagen del conjunto de entrenamiento

In [ ]:
plot_model_diff_vae(cvae,vae_train_dataset)

Como se puede observar, las imágenes no llegan a ser lo más fieles posibles a la imagen original. La razón principal es que, debido al tamaño de las imágenes, es necesario un espacio latente mucho más grande. El problema es que, si se aumenta el espacio latente, las dimensiones del modelo crecen exponencialmente, lo que lleva a problemas con la memoria.

### 2.2.4 Creación del modelo V2 <a id="vaecmv2"></a>

Vamos a probar otra solución: Separar el dataset en parches de 64x64x3 y así aumentar el espacio latente:

Función que, pasándole un dataset y un tamaño de parche, extrae cada parche y lo convierte en una canalización de dataset:

In [ ]:
def extract_patches(images_dataset, patch_size):

    patch_height, patch_width, _ = patch_size
    strides = [1, patch_height, patch_width, 1]

    def extract_patches_from_image(image):

        patches = tf.image.extract_patches(images=image,
                                           sizes=[1, patch_height, patch_width, 1],
                                           strides=strides,
                                           rates=[1, 1, 1, 1],
                                           padding='VALID')

        patches = tf.reshape(patches, (-1, patch_height, patch_width, 3))
        return patches

    patches_dataset = images_dataset.map(extract_patches_from_image,
                                         num_parallel_calls=tf.data.experimental.AUTOTUNE)

    # Concatena los parches
    patches_dataset = patches_dataset.unbatch()
    
    return patches_dataset


In [ ]:
vae_train_dataset_patches = extract_patches(vae_train_dataset, (64,64,3)).batch(30)
vae_valid_dataset_patches = extract_patches(vae_valid_dataset, (64,64,3)).batch(1)

Se muestran los resultados:

In [ ]:
plot_dataset(vae_train_dataset_patches,dim=1)

Creación del modelo. En esta ocasión se ha optado por reducir la dimensionalidad del modelo ya que las imágenes son mucho más pequeñas:

In [ ]:
def CVAE_PATCH(latent_dim):

  #Encoder
  encoder_input = tf.keras.layers.Input(shape=(64,64,3))
  
  x = tf.keras.layers.Conv2D(filters=32,  kernel_size=3,  activation='relu', padding='same')(encoder_input) # 64
  x = tf.keras.layers.Conv2D(filters=64,  kernel_size=3,  activation='relu', padding='same')(x) # 64
  x = tf.keras.layers.Conv2D(filters=128,  kernel_size=3, strides=2, activation='relu', padding='same')(x) # 32
  x = tf.keras.layers.Conv2D(filters=256, kernel_size=3,  activation='relu', padding='same')(x) # 32

  s = bk.int_shape(x)
  x = tf.keras.layers.Flatten()(x)
  
  z_mean = tf.keras.layers.Dense(latent_dim)(x)
  z_log_var = tf.keras.layers.Dense(latent_dim)(x)

  @tf.function
  def sampling(args):
      z_mean, z_log_var = args
      epsilon = bk.random_normal(shape=(bk.shape(z_mean)[0], latent_dim))
      return z_mean + bk.exp(z_log_var / 2) * epsilon

  # Reparameterization trick
  z = tf.keras.layers.Lambda(sampling)([z_mean, z_log_var])
  
  encoder = tf.keras.Model(encoder_input,[z_mean,z_log_var,z], name="encoder")

  #Decoder
  decoder_input = tf.keras.layers.Input(shape=bk.int_shape(z)[1:])
  x = tf.keras.layers.Dense(np.prod(s[1:]), activation='relu')(decoder_input)
  x = tf.keras.layers.Reshape(s[1:])(x)
  
  x = tf.keras.layers.Conv2DTranspose(filters=256, kernel_size=3, padding='same', activation='relu')(x) # 32
  x = tf.keras.layers.Conv2DTranspose(filters=128, kernel_size=3, strides=2, padding='same', activation='relu')(x) # 64
  x = tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=3, padding='same', activation='relu')(x) # 64
  x = tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=3, padding='same', activation='relu')(x) # 64
  x = tf.keras.layers.Conv2DTranspose(filters=3, kernel_size=3, strides=1, activation='sigmoid', padding='same')(x)

  decoder = tf.keras.models.Model(decoder_input,x, name="decoder")
  decoder_output = decoder(encoder(encoder_input)[2])

  vae = tf.keras.models.Model(encoder_input, decoder_output, name = "vae")

  reconstruction_loss = tf.keras.losses.mse(bk.flatten(encoder_input), bk.flatten(decoder_output))
  reconstruction_loss *= 64 * 64 * 3
  kl_loss = -0.5 * bk.sum(1 + z_log_var - bk.square(z_mean) - bk.exp(z_log_var), axis=1)
  B = 1000   
  vae_loss = bk.mean(B * reconstruction_loss + kl_loss)
  vae.add_loss(vae_loss)
  vae.add_metric(kl_loss, name="kl_loss")
  vae.add_metric(reconstruction_loss, name="reconstruction_loss")
  vae.compile(optimizer='adam')

  return vae

Se coge un espacio latente de 200:

In [ ]:
cvae_patch = CVAE_PATCH(200)

### 2.2.5 Entrenamiento del modelo <a id="vaeemv2"></a>

In [ ]:
cvae_history = cvae_patch.fit(vae_train_dataset_patches,shuffle=True,epochs=15,validation_data=vae_valid_dataset_patches)

In [ ]:
cvae_patch.save("Checkpoints/VAE_PATCH/VAE_128_model")

Se muestra el entrnamiento:

In [ ]:
pd.DataFrame(cvae_history.history).plot()
plt.xlabel('Epoch num.')
plt.show()

### 2.2.6 Resultados del entrenamiento V2 <a id="vaermv2"></a>

Se muestran los resultados:

In [ ]:
plot_model_diff_vae(cvae_patch,vae_valid_dataset_patches)

In [ ]:
plot_model_diff_vae(cvae_patch,vae_train_dataset_patches)

Como se puede ver, las imágenes pierden un poco el color y dejan de estar tan pixelados.

Se va ha probar a restaurar una imagen entera:

Función que pasándole un modelo y un tensor de imagen, separa en parches la imagen, predice cada parche y la reconstruye mostrándola por pantalla:

In [ ]:
def vae_patch_reconstruct(model, single_image,title="",figsize=(15, 15)):

    # Divide la imagen en parches de 48x48x3
    patches = tf.image.extract_patches(
        images=single_image,  
        sizes=[1, 64, 64, 1], 
        strides=[1, 64, 64, 1], 
        rates=[1, 1, 1, 1], 
        padding='VALID'  
    )

    patches = tf.reshape(patches, [-1, 64, 64, 3])

    #Se predicen los parches:
    reconstructed_patches = model.predict(patches,verbose=0)

    reconstructed_image = np.zeros((1024, 1024, 3))

    # Coloca cada parche en la imagen reconstruida
    patch_size = 64
    num_patches_per_row = 1024 // patch_size
    for i in range(len(reconstructed_patches)):
        row = i // num_patches_per_row
        col = i % num_patches_per_row
        reconstructed_image[row*patch_size:(row+1)*patch_size, col*patch_size:(col+1)*patch_size, :] = reconstructed_patches[i]

    plt.figure(figsize=figsize)

    plt.subplot(1, 2, 1)
    plt.imshow(single_image.numpy()[0]) 
    plt.title('Input')

    plt.subplot(1, 2, 2)

    plt.imshow(reconstructed_image) 
    plt.title('Prediction')

    plt.suptitle(title)
    plt.subplots_adjust(top=1.3)

    plt.show()

Se comparan las imágenes:

In [ ]:
single_image = vae_valid_dataset.take(1)
single_image = next(iter(single_image))

vae_patch_reconstruct(cvae_patch,single_image)

Se prueba una imagen con ruido para ver cómo de bien quita el ruido:

In [ ]:
input, target = next(iter(high_res_valid_dataset))

vae_patch_reconstruct(cvae_patch,input)

Como se puede ya observar, el modelo VAE (siendo entrenado de forma normal sin alterar la imagen de entrada como se ha hecho en el autoencoder), quita un poco de ruido pero no elimina la mayoría. Además, altera el color de las imágenes (quitando tonos verdes y azules) y haciendo que estas dejen de ser muy fiables.

## 2.3 GAN (Sin U-Net) <a id="gan"></a>

### Introducción a las redes GAN <a id="gani"></a>

Las redes GAN *(Generative Adversarial Networks)* son un tipo de arquitectura de red de neuronas. Este tipo de arquitectura consta de dos partes: la primera parte es el *generador* que, como su nombre indica, se encarga de generar imágenes y la segunda es el discriminador que se encarga de diferenciar si las imágenes son reales o no. Estas dos redes compiten por obtener los mejores resultados. El concepto de estas redes es parecido al algoritmo minimax, el discriminador y el generador intentan minimizar su valor de pérdida y maximizar la de su oponente.

![Basic Gan](./Images/Basic_GAN.png)

La fórmula general de las redes GAN parte de la fórmula de la entropía cruzada binaria  o *binary cross entropy*:<br><br>
- $L = - \varSigma \space yln(\widehat{y}) + (1-y)*ln(1-\widehat{y})$

- Si $y = 1 ; \space \widehat{y} = D(x) => L = ln[D(x)]$<br>

- Si $y = 0 ; \space \widehat{y} = D(G(z)) => L = ln[1 - D(G(z))]$ <br>

- Entonces: $L = ln[D(x)] + ln[1 - D(G(z))]$

Si aplicamos la esperanza en toda la igualdad nos queda:

- $E(L) = E(ln[D(x)]) + E(ln[1-D(G(z))])$

Sabiendo las propiedades de la esperanza para valores Continuos:

- $E[X] = \int_{R} Xf(X)dx$ => $ \textcolor{#FF33F3}{\int P_{datos}(x)} (ln[D(x)])\textcolor{#FF33F3}{dx} + \textcolor{#FF33F3}{\int P_{z}(z)}(ln[1-D(G(z))])\textcolor{#FF33F3}{dz}$

Las expresiones resaltadas se convierten en la función de valor:

- $min_G max_D V(G,D) = \textcolor{#FF33F3}{E_{x\text{\textasciitilde}P_{datos}}}[ln(D(x))] + \textcolor{#FF33F3}{E_{z\text{\textasciitilde}Pz}}[ln(1-D(G(z)))]$


### 2.3.1 Declaración de los Generadores <a id="dgan"></a>

Se declaran las funciones de downsample y upsample que se utilizarán mas adelante:

In [ ]:
def downsample(filters, size, apply_batchnorm=True):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
      tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False))

  if apply_batchnorm:
    result.add(tf.keras.layers.BatchNormalization())

  result.add(tf.keras.layers.LeakyReLU())

  return result

In [ ]:
def upsample(filters, size, apply_dropout=False):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
    tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                    padding='same',
                                    kernel_initializer=initializer,
                                    use_bias=False))

  result.add(tf.keras.layers.BatchNormalization())

  if apply_dropout:
      result.add(tf.keras.layers.Dropout(0.5))

  result.add(tf.keras.layers.ReLU())

  return result


En el caso de los generadores siguen la siguiente estructura:

![generador normal](./Images/Generador_normal.png)

In [ ]:
def Generator_256():

  inputs = tf.keras.layers.Input(shape=[256, 256, 3])

  #Downsample
  x = tf.keras.layers.Conv2D(64, 4, strides=2, padding='same', activation='relu')(inputs) #128
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Conv2D(128, 4, strides=2, padding='same', activation='relu')(x) # 64
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Conv2D(256, 4, strides=2, padding='same', activation='relu')(x) # 32
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Conv2D(512, 4, strides=2, padding='same', activation='relu')(x) # 16
  x = tf.keras.layers.BatchNormalization()(x)

  #Upsample
  x = tf.keras.layers.Conv2DTranspose(512, 4, strides=2, padding='same', activation='relu')(x) # 32
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Conv2DTranspose(256, 4, strides=2, padding='same', activation='relu')(x) # 64
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Conv2DTranspose(128, 4, strides=2, padding='same', activation='relu')(x) # 128
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Conv2DTranspose(64, 4, strides=2, padding='same', activation='relu')(x) # 256
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Conv2DTranspose(32, 4, strides=2, padding='same', activation='relu')(x) # 512
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Conv2DTranspose(16, 4, strides=2, padding='same', activation='relu')(x) # 1024
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Conv2DTranspose(3, 4, strides=1, padding='same', activation='sigmoid')(x) # 1024

  return tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
generator_256 = Generator_256()
tf.keras.utils.plot_model(generator_256, show_shapes=True, dpi=64)

In [ ]:
def Generator_512():

  inputs = tf.keras.layers.Input(shape=[512, 512, 3])

  #Downsample
  x = tf.keras.layers.Conv2D(64, 4, strides=2, padding='same', activation='relu')(inputs) #256
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Conv2D(128, 4, strides=2, padding='same', activation='relu')(x) # 128
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Conv2D(256, 4, strides=2, padding='same', activation='relu')(x) # 64
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Conv2D(512, 4, strides=2, padding='same', activation='relu')(x) # 32
  x = tf.keras.layers.BatchNormalization()(x)

  #Upsample
  x = tf.keras.layers.Conv2DTranspose(512, 4, strides=2, padding='same', activation='relu')(x) # 64
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Conv2DTranspose(256, 4, strides=2, padding='same', activation='relu')(x) # 128
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Conv2DTranspose(128, 4, strides=2, padding='same', activation='relu')(x) # 256
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Conv2DTranspose(64, 4, strides=2, padding='same', activation='relu')(x) # 512
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Conv2DTranspose(32, 4, strides=2, padding='same', activation='relu')(x) # 512
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Conv2DTranspose(3, 4, strides=1, padding='same', activation='sigmoid')(x) # 1024


  return tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
generator_512 = Generator_512()
tf.keras.utils.plot_model(generator_512, show_shapes=True, dpi=64)

In [ ]:
def Generator_1024():

  inputs = tf.keras.layers.Input(shape=[1024, 1024, 3])

  #Downsample
  x = tf.keras.layers.Conv2D(64, 4, strides=2, padding='same', activation='relu')(inputs) #512
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Conv2D(128, 4, strides=2, padding='same', activation='relu')(x) # 256
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Conv2D(256, 4, strides=2, padding='same', activation='relu')(x) # 128
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Conv2D(512, 4, strides=2, padding='same', activation='relu')(x) # 64
  x = tf.keras.layers.BatchNormalization()(x)

  #Upsample
  x = tf.keras.layers.Conv2DTranspose(512, 4, strides=2, padding='same', activation='relu')(x) # 128
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Conv2DTranspose(256, 4, strides=2, padding='same', activation='relu')(x) # 256
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Conv2DTranspose(128, 4, strides=2, padding='same', activation='relu')(x) # 512
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Conv2DTranspose(64, 4, strides=2, padding='same', activation='relu')(x) # 512
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Conv2DTranspose(3, 4, strides=1, padding='same', activation='sigmoid')(x) # 10224


  return tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
generator_1024 = Generator_1024()
tf.keras.utils.plot_model(generator_1024, show_shapes=True, dpi=64)

### 2.3.2 Declaración de las funciones de pérdida <a id="dfp"></a>

En cuanto a la función de pérdida del generador es de la siguiente manera:

![Generator_loss](Images\losses_GAN_Generador.png)

In [ ]:
LAMBDA = 100

In [ ]:
def generator_loss(loss_object,disc_generated_output, gen_output, target):
  gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)

  # Mean absolute error
  l1_loss = tf.reduce_mean(tf.abs(target - gen_output))

  total_gen_loss = gan_loss + (LAMBDA * l1_loss)

  return total_gen_loss, gan_loss, l1_loss

En cuanto a la pérdida del Discriminador es de la siguiente manera:

![Discriminator loss](Images\losses_GAN_Discriminador.png)

In [ ]:
def discriminator_loss(loss_object,disc_real_output, disc_generated_output):
  real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)

  generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)

  total_disc_loss = real_loss + generated_loss

  return total_disc_loss

### 2.3.3 Declarción del discriminador <a id="dd"></a>

La estructura del discriminador es un PatchGAN convolucional como en el artículo de Pix2Pix:

1. El discriminador recoge dos entradas y las concatena.
2. Hace cuatro downsample con Conv2d->BatchNorm->LeakyRelu cada uno.
3. Se amplia y se aumenta el tamaño de la capa y se hace un ultimo Conv2d->BatchNorm->Leaky Relu y se vuelve a reducir.
4. La capa final tiene un solo filtro

In [ ]:
def Discriminator():
  initializer = tf.random_normal_initializer(0., 0.02)

  inp = tf.keras.layers.Input(shape=[1024, 1024, 3], name='input_image')
  tar = tf.keras.layers.Input(shape=[1024, 1024, 3], name='target_image')

  #Se concatenan las dos entradas
  x = tf.keras.layers.concatenate([inp, tar])  # (batch_size, 1024, 1024, channels*2) 

  down1 = downsample(64, 4, False)(x)  # (batch_size, 512, 512, 64)
  down2 = downsample(128, 4)(down1)  # (batch_size, 256, 256, 128)
  down3 = downsample(256, 4)(down2)  # (batch_size, 128, 128, 256)
  down4 = downsample(512, 4)(down3)  # (batch_size, 64, 64, 512)

  # Se añade relleno
  zero_pad1 = tf.keras.layers.ZeroPadding2D()(down4)  # (batch_size, 66, 66, 512)
  conv = tf.keras.layers.Conv2D(512, 4, strides=1,
                                kernel_initializer=initializer,
                                use_bias=False)(zero_pad1)  # (batch_size, 63, 63, 1024)

  batchnorm1 = tf.keras.layers.BatchNormalization()(conv)

  leaky_relu = tf.keras.layers.LeakyReLU()(batchnorm1)

  zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu)  # (batch_size, 65, 65, 1024)

  last = tf.keras.layers.Conv2D(1, 4, strides=1,
                                kernel_initializer=initializer)(zero_pad2)  # (batch_size, 62, 62, 1)

  return tf.keras.Model(inputs=[inp, tar], outputs=last)

In [ ]:
discriminator_256 = Discriminator()
discriminator_512 = Discriminator()
discriminator_1024 = Discriminator()

tf.keras.utils.plot_model(discriminator_256, show_shapes=True, dpi=64)

### 2.3.4 Declaración de las funciones de entenamiento <a id="dfe"></a>

En el entrenamiento se entrena primero el generador y luego el discriminador. Estos NO pueden entrenarse a la vez.

In [ ]:
def train_step(generator,discriminator,generator_optimizer,discriminator_optimizer,loss_object,input_image, target):
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    
    #Entrenamiento del generador
    gen_output = generator(input_image, training=True)

    # Uso el método bilinear para reescalar la imagen de input
    bilinear_input_image = tf.image.resize(input_image,size=(1024,1024),method="bilinear")

    #Entrenamiento del Discriminador
    disc_real_output = discriminator([bilinear_input_image, target], training=True)
    disc_generated_output = discriminator([bilinear_input_image, gen_output], training=True)

    #Se genera la pérdida total del generador y del discriminador:
    gen_total_loss, gen_gan_loss, gen_l1_loss = generator_loss(loss_object, disc_generated_output, gen_output, target)
    disc_loss = discriminator_loss(loss_object, disc_real_output, disc_generated_output)

  #Se aplican los gradientes a las redes
  generator_gradients = gen_tape.gradient(gen_total_loss,
                                          generator.trainable_variables)
  discriminator_gradients = disc_tape.gradient(disc_loss,
                                               discriminator.trainable_variables)
  #Se aplican los gradientes a los optimizers
  generator_optimizer.apply_gradients(zip(generator_gradients,
                                          generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients,
                                              discriminator.trainable_variables))

En la funcion de entrenamiento en cada step se saca un batch de imágenes y se prepara el train_step. Cada 5k steps se guarda el checkpoint para poder restaurar la imagen cuando se desee.

In [ ]:
def fit(generator ,discriminator, train_ds,generator_optimizer, discriminator_optimizer , checkpoint, checkpoint_prefix,loss_object, steps):

  #Tiempo de inicio
  start = time.time()

  #Se itera y se cogen N steps de imágenes
  for step, (input_image, target) in train_ds.repeat().take(steps).enumerate():

    # Limpia la pantalla cuando hay 1k pasos e imprime el tiempo que ha tardado
    with tf.device('/CPU:0'):
      if (step) % 1000 == 0:
        display.clear_output(wait=True)

        if step != 0:
          print(f'Time taken for 1000 steps: {time.time()-start:.2f} sec\n')

        start = time.time()
        print(f"Step: {step//1000}k")

    #Entrenamiento de los modelos
    train_step(generator, discriminator,generator_optimizer, discriminator_optimizer,loss_object,input_image,target)

    # Cada 10 pasos pone un punto
    with tf.device('/CPU:0'):
      if (step+1) % 10 == 0:
        print('.', end='', flush=True)


    # Guarda el checkpoint cada 5k
    with tf.device('/CPU:0'):
      if (step + 1) % 5000 == 0:
        checkpoint.save(file_prefix=checkpoint_prefix)


### 2.3.5 Entrenamiento y comprobación del modelo <a id="ganecm"></a>

#### 2.3.5.1 Entrenamiento de 256->1024

Se crea un checkpoint en caso de que se quiera reaunudar el entrenamiento o se quiera recuperar una etapa anterior:

In [ ]:
generator_optimizer_256 = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer_256 = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
loss_object_256 = tf.keras.losses.BinaryCrossentropy(from_logits=True)
checkpoint_dir = './Checkpoints/GAN/ckpt_gan_256/'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint_256 = tf.train.Checkpoint(generator_optimizer=generator_optimizer_256,
                                 discriminator_optimizer=discriminator_optimizer_256,
                                 generator= generator_256,
                                 discriminator= discriminator_256)

Se entrena el modelo:

In [ ]:
fit(generator_256,discriminator_256,low_res_train_dataset,generator_optimizer_256,discriminator_optimizer_256,checkpoint_256,checkpoint_prefix,loss_object_256,40000)

In [ ]:
generator_256.save("Checkpoints/GAN/generator_256_model")
discriminator_256.save("Checkpoints/GAN/discriminator_256_model")

Se muestra el resultado del entrenamiento

In [ ]:
plot_model_diff(generator_256,low_res_valid_dataset)

#### 2.3.5.2 Entrenamiento de 512->1024

In [ ]:
generator_optimizer_512 = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer_512 = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
loss_object_512 = tf.keras.losses.BinaryCrossentropy(from_logits=True)
checkpoint_dir = './Checkpoints/GAN/ckpt_gan_512/'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint_512 = tf.train.Checkpoint(generator_optimizer=generator_optimizer_512,
                                 discriminator_optimizer=discriminator_optimizer_512,
                                 generator= generator_512,
                                 discriminator= discriminator_512)

In [ ]:
fit(generator_512,discriminator_512,med_res_train_dataset,generator_optimizer_512,discriminator_optimizer_512,checkpoint_512,checkpoint_prefix,loss_object_512,30000)

In [ ]:
generator_512.save("Checkpoints/GAN/generator_512_model")
discriminator_512.save("Checkpoints/GAN/discriminator_512_model")

In [ ]:
plot_model_diff(generator_512,med_res_valid_dataset)

#### 2.3.5.3 Entrenamiento de 1024->1024

In [ ]:
generator_optimizer_1024 = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer_1024 = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
loss_object_1024 = tf.keras.losses.BinaryCrossentropy(from_logits=True)
checkpoint_dir = './Checkpoints/GAN/ckpt_gan_1024/'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint_1024 = tf.train.Checkpoint(generator_optimizer=generator_optimizer_1024,
                                 discriminator_optimizer=discriminator_optimizer_1024,
                                 generator= generator_1024,
                                 discriminator= discriminator_1024)

In [ ]:
fit(generator_1024,discriminator_1024,high_res_train_dataset,generator_optimizer_1024,discriminator_optimizer_1024,checkpoint_1024,checkpoint_prefix,loss_object_1024,10000)

In [ ]:
generator_1024.save("Checkpoints/GAN/generator_1024_model")
discriminator_1024.save("Checkpoints/GAN/discriminator_1024_model")

In [ ]:
plot_model_diff(generator_1024,high_res_valid_dataset)

Como se puede observar, este modelo no consigue mantener el color y con algunos errores en la imagen.

## 2.4 GAN (U-Net o cGAN o pix2pix) <a id="cgan"></a>

 ### 2.4.1 Declaración de los Generadores <a id="dunet"></a>

Esta Red GAN está basada en el documento pix2pix y su implementación en la documentación de tensorflow.

Lo único que cambia respecto de las anteriores redes es el generador. Los generadores son una U-NET un poco modificada:

- El modelo de x4 tiene 7 capas de downsample y 6 capas de upsample junto con una capa extra de Conv2DTranspose y una operación de profundidad a espacio.
- El modelo de x2 tiene 8 capas de downsample y 7 capas de upsample junto con una operación de profundidad a espacio.
- El modelo de x1 tiene 9 capas de downsample y 8 capas de upsample sin operaciones extra.


![resumen U-NET](./Images/Resumen_U-NET.png)

#### 2.4.1.1 Generador 256 -> 1024

In [ ]:
def unet_Generator_256():
  inputs = tf.keras.layers.Input(shape=[256, 256, 3])

  #Downsample
  down_stack = [
    downsample(64, 4, apply_batchnorm=False),  # (batch_size, 128, 128, 64)
    downsample(128, 4),  # (batch_size, 64, 64, 128)
    downsample(256, 4),  # (batch_size, 32, 32, 256)
    downsample(512, 4),  # (batch_size, 16, 16, 512)
    downsample(512, 4),  # (batch_size, 8, 8, 512)
    downsample(512, 4),  # (batch_size, 4, 4, 512)
    downsample(512, 4),  # (batch_size, 2, 2, 512)
  ]

  #Upsample
  up_stack = [
    upsample(512, 4, apply_dropout=True),  # (batch_size, 4, 4, 1024)
    upsample(512, 4, apply_dropout=True),  # (batch_size, 8, 8, 1024)
    upsample(512, 4),  # (batch_size, 16, 16, 1024)
    upsample(256, 4),  # (batch_size, 32, 32, 512)
    upsample(128, 4),  # (batch_size, 64, 64, 256)
    upsample(64, 4),  # (batch_size, 128, 128, 64)
  ]

  # Última capa de convolucion:
  initializer = tf.random_normal_initializer(0., 0.02)
  last = tf.keras.layers.Conv2DTranspose(3, 4,
                                         strides=2,
                                         padding='same',
                                         kernel_initializer=initializer,
                                         activation='tanh')  # (batch_size, 256, 256, 3)

  x = inputs

  # Conecta las capas de downsample:
  skips = []
  for down in down_stack:
    x = down(x)
    skips.append(x)

  skips = reversed(skips[:-1])

  # Conecta y concatena las capas de downsample con las de upsample
  for up, skip in zip(up_stack, skips):
    x = up(x)
    x = tf.keras.layers.Concatenate()([x, skip])

  #Escalado del modelo:
  x = tf.keras.layers.Conv2DTranspose(32, 4, strides=2, padding='same', kernel_initializer=initializer, use_bias=False)(x)
  x = tf.keras.layers.Conv2D(filters=16, kernel_size=3, strides=1, padding='SAME')(x)
  x = tf.keras.layers.Lambda(lambda x : tf.nn.depth_to_space(x,2))(x)

  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)


In [ ]:
unet_generator_256 = unet_Generator_256()
tf.keras.utils.plot_model(unet_generator_256, show_shapes=True, dpi=64)

#### 2.4.1.2 Generador 512 -> 1024

In [ ]:
def unet_Generator_512():
  inputs = tf.keras.layers.Input(shape=[512, 512, 3])

  #Downsample
  down_stack = [
    downsample(64, 4, apply_batchnorm=False),  # (batch_size, 256, 256, 64)
    downsample(128, 4),  # (batch_size, 128, 128, 128)
    downsample(256, 4),  # (batch_size, 64, 64, 256)
    downsample(512, 4),  # (batch_size, 32, 32, 512)
    downsample(512, 4),  # (batch_size, 16, 16, 512)
    downsample(512, 4),  # (batch_size, 8, 8, 512)
    downsample(512, 4),  # (batch_size, 4, 4, 512)
    downsample(512, 4),  # (batch_size, 2, 2, 512)
  ]

  #Upsample
  up_stack = [
    upsample(512, 4, apply_dropout=True),  # (batch_size, 4, 4, 1024)
    upsample(512, 4, apply_dropout=True),  # (batch_size, 8, 8, 1024)
    upsample(512, 4),  # (batch_size, 16, 16, 1024)
    upsample(512, 4),  # (batch_size, 32, 32, 1024)
    upsample(256, 4),  # (batch_size, 64, 64, 512)
    upsample(128, 4),  # (batch_size, 128, 128, 256)
    upsample(64, 4),  # (batch_size, 256, 256, 128)
  ]

  # Última capa de convolucion:
  initializer = tf.random_normal_initializer(0., 0.02)
  last = tf.keras.layers.Conv2DTranspose(3, 4,
                                         strides=2,
                                         padding='same',
                                         kernel_initializer=initializer,
                                         activation='tanh')  # (batch_size, 256, 256, 3)

  x = inputs

  # Conecta las capas de downsample:
  skips = []
  for down in down_stack:
    x = down(x)
    skips.append(x)

  skips = reversed(skips[:-1])

  # Conecta y concatena las capas de downsample con las de upsample
  for up, skip in zip(up_stack, skips):
    x = up(x)
    x = tf.keras.layers.Concatenate()([x, skip])

  #Escalado del modelo:
  x = tf.keras.layers.Conv2D(filters= (512 * 4), kernel_size=3, strides=1, padding='SAME')(x)
  x = tf.keras.layers.Lambda(lambda x : tf.nn.depth_to_space(x,2))(x)


  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)


In [ ]:
unet_generator_512 = unet_Generator_512()
tf.keras.utils.plot_model(unet_generator_512, show_shapes=True, dpi=64)

#### 2.4.1.3 Generador 1024->1024

In [ ]:
def unet_Generator_1024():
  inputs = tf.keras.layers.Input(shape=[1024, 1024, 3])

  #Downsample
  down_stack = [
    downsample(64, 4, apply_batchnorm=False),  # (batch_size, 512, 512, 64)
    downsample(128, 4),  # (batch_size, 256, 256, 128)
    downsample(256, 4),  # (batch_size, 128, 128, 256)
    downsample(512, 4),  # (batch_size, 64, 64, 512)
    downsample(512, 4),  # (batch_size, 32, 32, 512)
    downsample(512, 4),  # (batch_size, 16, 16, 512)
    downsample(512, 4),  # (batch_size, 8, 8, 512)
    downsample(512, 4),  # (batch_size, 4, 4, 512)
    downsample(512, 4),  # (batch_size, 2, 2, 512)
  ]

  #Upsample
  up_stack = [
    upsample(512, 4, apply_dropout=True),  # (batch_size, 4, 2, 1024)
    upsample(512, 4, apply_dropout=True),  # (batch_size, 8, 4, 1024)
    upsample(512, 4, apply_dropout=True),  # (batch_size, 16, 8, 1024)
    upsample(512, 4),  # (batch_size, 32, 32, 1024)
    upsample(512, 4),  # (batch_size, 64, 64, 1024)
    upsample(256, 4),  # (batch_size, 128, 128, 512)
    upsample(128, 4),  # (batch_size, 256, 256, 256)
    upsample(64, 4),  # (batch_size, 512, 512, 128)
  ]

  # Última capa de convolucion:
  initializer = tf.random_normal_initializer(0., 0.02)
  last = tf.keras.layers.Conv2DTranspose(3, 4,
                                         strides=2,
                                         padding='same',
                                         kernel_initializer=initializer,
                                         activation='tanh')  # (batch_size, 256, 256, 3)

  x = inputs

  # Conecta las capas de downsample:
  skips = []
  for down in down_stack:
    x = down(x)
    skips.append(x)

  skips = reversed(skips[:-1])

  # Conecta y concatena las capas de downsample con las de upsample
  for up, skip in zip(up_stack, skips):
    x = up(x)
    x = tf.keras.layers.Concatenate()([x, skip])

  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)


In [ ]:
unet_generator_1024 = unet_Generator_1024()
tf.keras.utils.plot_model(unet_generator_1024, show_shapes=True, dpi=64)

### 2.4.2 Declaración de los discriminadores y entrenamiento <a id="unetde"></a>

In [ ]:
unet_discriminator_256 = Discriminator()
unet_discriminator_512 = Discriminator()
unet_discriminator_1024 = Discriminator()

tf.keras.utils.plot_model(unet_discriminator_256, show_shapes=True, dpi=64)

#### 2.4.2.1 Entrenamiento de 256->1024

In [ ]:
unet_generator_optimizer_256 = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
unet_discriminator_optimizer_256 = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
unet_loss_object_256 = tf.keras.losses.BinaryCrossentropy(from_logits=True)
checkpoint_dir = './Checkpoints/unet_GAN/ckpt_gan_256/'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
unet_checkpoint_256 = tf.train.Checkpoint(generator_optimizer=unet_generator_optimizer_256,
                                 discriminator_optimizer=unet_discriminator_optimizer_256,
                                 generator=unet_generator_256,
                                 discriminator=unet_discriminator_256)

In [ ]:
fit(unet_generator_256,unet_discriminator_256,low_res_train_dataset,unet_generator_optimizer_256,unet_discriminator_optimizer_256,unet_checkpoint_256,checkpoint_prefix,unet_loss_object_256,40000)

In [ ]:
unet_generator_256.save("Checkpoints/unet_GAN/generator_256_model")
unet_discriminator_256.save("Checkpoints/unet_GAN/discriminator_256_model")

In [ ]:
plot_model_diff(unet_generator_256,low_res_valid_dataset)

#### 2.4.2.2 Entrenamiento de 512->1024

In [ ]:
unet_generator_optimizer_512 = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
unet_discriminator_optimizer_512 = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
unet_loss_object_512 = tf.keras.losses.BinaryCrossentropy(from_logits=True)
checkpoint_dir = './Checkpoints/unet_GAN/ckpt_gan_512/'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
unet_checkpoint_512 = tf.train.Checkpoint(generator_optimizer=unet_generator_optimizer_512,
                                 discriminator_optimizer=unet_discriminator_optimizer_512,
                                 generator=unet_generator_512,
                                 discriminator=unet_discriminator_512)

In [ ]:
fit(unet_generator_512,unet_discriminator_512,med_res_train_dataset,unet_generator_optimizer_512,unet_discriminator_optimizer_512,unet_checkpoint_512,checkpoint_prefix,unet_loss_object_512,20000)

In [ ]:
unet_generator_512.save("Checkpoints/unet_GAN/generator_512_model")
unet_discriminator_512.save("Checkpoints/unet_GAN/discriminator_512_model")

In [ ]:
plot_model_diff(unet_generator_512,med_res_valid_dataset)

#### 2.4.2.1 Entrenamiento de 1024->1024

In [ ]:
unet_generator_optimizer_1024 = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
unet_discriminator_optimizer_1024 = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
unet_loss_object_1024 = tf.keras.losses.BinaryCrossentropy(from_logits=True)
checkpoint_dir = './Checkpoints/unet_GAN/ckpt_gan_1024/'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
unet_checkpoint_1024 = tf.train.Checkpoint(generator_optimizer=unet_generator_optimizer_1024,
                                 discriminator_optimizer=unet_discriminator_optimizer_1024,
                                 generator=unet_generator_1024,
                                 discriminator=unet_discriminator_1024)

In [ ]:
fit(unet_generator_1024,unet_discriminator_1024,high_res_train_dataset,unet_generator_optimizer_1024,unet_discriminator_optimizer_1024,unet_checkpoint_1024,checkpoint_prefix,unet_loss_object_1024,10000)

In [ ]:
unet_generator_1024.save("Checkpoints/unet_GAN/generator_1024_model")
unet_discriminator_1024.save("Checkpoints/unet_GAN/discriminator_1024_model")

In [ ]:
plot_model_diff(unet_generator_1024,high_res_valid_dataset)

Este modelo lo hace mejor que los anteriores, pero sigue dejando un poco de ruido en la imagen.

## 2.5 EDSR_GAN <a id="edsrgan"></a>

### 2.5.1 Creación del generador con arquitectura EDSR <a id="edsr"></a>

En la arquitectura EDSR los bloques residuales NO tienen batchnormalization ya que le puede quitar calidad a la imagen.

Este tipo de arquitextura consta de B bloques residuales con F filtros por capa de convolución.

<img src="./Images/EDSR_structure.png">

Nota: Imagen sacada de el paper: https://arxiv.org/pdf/1707.02921

In [ ]:

def EDSR(scale, input_shape,B=8, F=256, scale_factor=0.1):

    xinput = tf.keras.layers.Input(shape=input_shape)
    # Primer Conv2d
    xlast = x = tf.keras.layers.Conv2D(filters=F, kernel_size=3, strides=1, padding='SAME')(xinput)
    
    # Bloques residuales:
    def res_block(x):
        x1 = tf.keras.layers.Conv2D(filters= F, kernel_size=3, strides=1, padding='SAME', activation="relu")(x) # Conv2d + relu
        x2 = tf.keras.layers.Conv2D(filters= F, kernel_size=3, strides=1, padding='SAME')(x1) # Conv2d

        x2 = x2 * scale_factor # Mult
        output = tf.keras.layers.Add()([x2, x]) # Add
        return output
    
    # B ResBlocks
    for i in range(B):
        x = res_block(x)

    # Último Conv2d
    x = tf.keras.layers.Conv2D(filters=F, kernel_size=3, strides=1, padding='SAME')(x)
    # Último Add
    x = tf.keras.layers.Add()([x,xlast])

    # Upsample
    if scale == 2: # x2 512
        x = tf.keras.layers.Conv2D(filters= (16), kernel_size=3, strides=1, padding='SAME')(x)
        x = tf.keras.layers.Lambda(lambda x : tf.nn.depth_to_space(x,2))(x)
    elif scale == 4: # x4 256

        x = tf.keras.layers.Conv2D(filters= (32), kernel_size=3, strides=1, padding='SAME')(x)
        x = tf.keras.layers.Lambda(lambda x : tf.nn.depth_to_space(x,2))(x)

        x = tf.keras.layers.Conv2D(filters= (16), kernel_size=3, strides=1, padding='SAME')(x)
        x = tf.keras.layers.Lambda(lambda x : tf.nn.depth_to_space(x,2))(x)

    out = tf.keras.layers.Conv2D(filters=3, kernel_size=3, strides=1, padding='SAME')(x)

    return tf.keras.models.Model(xinput, out)


In [ ]:
def EDSR_x1(scale, input_shape,B=8, F=256, scale_factor=0.1):

    xinput = tf.keras.layers.Input(shape=input_shape)
    
    #Downsample
    x = downsample(16,4,apply_batchnorm=False)(xinput)
    x = downsample(32,4,apply_batchnorm=False)(x)

    xlast = x = tf.keras.layers.Conv2D(filters=F, kernel_size=3, strides=1, padding='SAME')(x)
    
    # Bloques residuales:
    def res_block(x):
        x1 = tf.keras.layers.Conv2D(filters= F, kernel_size=3, strides=1, padding='SAME', activation="relu")(x) # Conv2d + relu
        x2 = tf.keras.layers.Conv2D(filters= F, kernel_size=3, strides=1, padding='SAME')(x1) # Conv2d

        x2 = x2 * scale_factor # Mult
        output = tf.keras.layers.Add()([x2, x]) # Add
        return output
    
    # B ResBlocks
    for i in range(B):
        x = res_block(x)

    # Último Conv2d
    x = tf.keras.layers.Conv2D(filters=F, kernel_size=3, strides=1, padding='SAME')(x)
    # Último Add
    x = tf.keras.layers.Add()([x,xlast])

    # Upsample

    x = upsample(32,4)(x)
    x = upsample(16,4)(x)

    out = tf.keras.layers.Conv2D(filters=3, kernel_size=3, strides=1, padding='SAME')(x)

    return tf.keras.models.Model(xinput, out)


In [ ]:
edsr_generator_256 = EDSR(4,(256,256,3))
tf.keras.utils.plot_model(edsr_generator_256, show_shapes=True, dpi=64)

In [ ]:
edsr_generator_512 = EDSR(2,(512,512,3),B=8,F=128)
tf.keras.utils.plot_model(edsr_generator_512, show_shapes=True, dpi=64)

In [ ]:
edsr_generator_1024 = EDSR_x1(1,(1024,1024,3),B=8,F=256)
tf.keras.utils.plot_model(edsr_generator_1024, show_shapes=True, dpi=64)

### 2.5.2 Declaración de los discriminadores y entrenamiento <a id="edsrde"></a>

In [ ]:
edsr_discriminator_256 = Discriminator()
edsr_discriminator_512 = Discriminator()
edsr_discriminator_1024 = Discriminator()

#tf.keras.utils.plot_model(edsr_discriminator_256, show_shapes=True, dpi=64)

#### 2.5.2.1 Entrenamiento 256->1024

In [ ]:
edsr_generator_optimizer_256 = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
edsr_discriminator_optimizer_256 = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
edsr_loss_object_256 = tf.keras.losses.BinaryCrossentropy(from_logits=True)
checkpoint_dir = './Checkpoints/edsr_GAN/ckpt_gan_256/'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
edsr_checkpoint_256 = tf.train.Checkpoint(generator_optimizer=edsr_generator_optimizer_256,
                                 discriminator_optimizer=edsr_discriminator_optimizer_256,
                                 generator=edsr_generator_256,
                                 discriminator=edsr_discriminator_256)

In [ ]:
fit(edsr_generator_256,edsr_discriminator_256,low_res_train_dataset,edsr_generator_optimizer_256,edsr_discriminator_optimizer_256,edsr_checkpoint_256,checkpoint_prefix,edsr_loss_object_256,20000)

In [ ]:
edsr_generator_256.save("Checkpoints/edsr_GAN/generator_256_model")
edsr_discriminator_256.save("Checkpoints/edsr_GAN/discriminator_256_model")

In [ ]:
plot_model_diff(edsr_generator_256,low_res_valid_dataset)

#### 2.5.2.2 Entrenamiento 512->1024

In [ ]:
edsr_generator_optimizer_512 = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
edsr_discriminator_optimizer_512 = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
edsr_loss_object_512 = tf.keras.losses.BinaryCrossentropy(from_logits=True)
checkpoint_dir = './Checkpoints/edsr_GAN/ckpt_gan_512/'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
edsr_checkpoint_512 = tf.train.Checkpoint(generator_optimizer=edsr_generator_optimizer_512,
                                 discriminator_optimizer=edsr_discriminator_optimizer_512,
                                 generator=edsr_generator_512,
                                 discriminator=edsr_discriminator_512)

In [ ]:
fit(edsr_generator_512,edsr_discriminator_512,med_res_train_dataset,edsr_generator_optimizer_512,edsr_discriminator_optimizer_512,edsr_checkpoint_512,checkpoint_prefix,edsr_loss_object_512,20000)

In [ ]:
edsr_generator_512.save("Checkpoints/edsr_GAN/generator_512_model")
edsr_discriminator_512.save("Checkpoints/edsr_GAN/discriminator_512_model")

In [ ]:
plot_model_diff(edsr_generator_512,med_res_valid_dataset)

#### 2.5.2.3 Entrenamiento 1024->1024

In [ ]:
edsr_generator_optimizer_1024 = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
edsr_discriminator_optimizer_1024 = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
edsr_loss_object_1024 = tf.keras.losses.BinaryCrossentropy(from_logits=True)
checkpoint_dir = './Checkpoints/edsr_GAN/ckpt_gan_1024/'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
edsr_checkpoint_1024 = tf.train.Checkpoint(generator_optimizer=edsr_generator_optimizer_1024,
                                 discriminator_optimizer=edsr_discriminator_optimizer_1024,
                                 generator=edsr_generator_1024,
                                 discriminator=edsr_discriminator_1024)

In [ ]:
fit(edsr_generator_1024,edsr_discriminator_1024,high_res_train_dataset,edsr_generator_optimizer_1024,edsr_discriminator_optimizer_1024,edsr_checkpoint_1024,checkpoint_prefix,edsr_loss_object_1024,15000)

In [ ]:
edsr_generator_1024.save("Checkpoints/edsr_GAN/generator_1024_model")
edsr_discriminator_1024.save("Checkpoints/edsr_GAN/discriminator_1024_model")

In [ ]:
plot_model_diff(edsr_generator_1024,high_res_valid_dataset)

Como se puede ver, este modelo funciona muy bien con las imágenes de x2 y x4, pero con x1, al reducir la dimensionalidad del modelo, se pierde mucha información y por eso pierde colores.

# 3. Comparación de modelos <a id="comparacionmodelos"></a>

Se va ha proceder a analizar los resultados de entrenamiento de cada modelo separados por tres categorías:

- 256x256 a 1024x1024 (resolución x4)
- 512x512 a 1024x1024 (resolución x2)
- 1024x1024 a 1024x1024 (resolución x1)

Primero se cargan todos los modelos

## 3.1 Carga de modelos: <a id="cm"></a>

Autoencoder:

In [ ]:
autoencoder_256 = tf.keras.models.load_model('Checkpoints\Autoencoder/autoencoder_256_model')

autoencoder_512 = tf.keras.models.load_model('Checkpoints\Autoencoder/autoencoder_512_model')

autoencoder_1024 = tf.keras.models.load_model('Checkpoints\Autoencoder/autoencoder_1024_model')

VAE:

In [ ]:
def sampling(latent_dim):
    @tf.function
    def _sampling(args):
        z_mean, z_log_var = args
        epsilon = tf.keras.backend.random_normal(shape=(tf.shape(z_mean)[0], latent_dim))
        return z_mean + tf.keras.backend.exp(z_log_var / 2) * epsilon
    return _sampling


custom_object_vae = {'sampling': sampling(256)}

custom_object_vae_pathc = {'sampling': sampling(200)}


#Ambos modelos son de 1024x1024
vae = tf.keras.models.load_model('Checkpoints/VAE/VAE_1024_model',custom_objects=custom_object_vae)

vae_patch = tf.keras.models.load_model('Checkpoints\VAE_PATCH\VAE_64_model',custom_objects=custom_object_vae_pathc)

GAN Vanilla

In [ ]:
gan_256 = tf.keras.models.load_model('Checkpoints\GAN\generator_256_model')

gan_512 = tf.keras.models.load_model('Checkpoints\GAN\generator_512_model')

gan_1024 = tf.keras.models.load_model('Checkpoints\GAN\generator_1024_model')

GAN U-NET

In [ ]:
unet_gan_256 = tf.keras.models.load_model(r'Checkpoints\unet_GAN\generator_256_model')

unet_gan_512 = tf.keras.models.load_model(r'Checkpoints\unet_GAN\generator_512_model')

unet_gan_1024 = tf.keras.models.load_model(r'Checkpoints\unet_GAN\generator_1024_model')

GAN EDSR

In [ ]:
edsr_gan_256 = tf.keras.models.load_model(r'Checkpoints\edsr_GAN\generator_256_model')

edsr_gan_512 = tf.keras.models.load_model(r'Checkpoints\edsr_GAN\generator_512_model')

edsr_gan_1024 = tf.keras.models.load_model(r'Checkpoints\edsr_GAN\generator_1024_model')

# 3.2 Comparativa de imágenes <a id="ci"></a>

Se va ha proceder a comparar visualmente los resultados de cada modelo:

In [ ]:
names = ["autoencoder","gan","unet_gan","edsr_gan"]
model_list_256 = [autoencoder_256,gan_256,unet_gan_256,edsr_gan_256]
model_list_512 = [autoencoder_512,gan_512,unet_gan_512,edsr_gan_512]
model_list_1024 = [autoencoder_1024,gan_1024,unet_gan_1024,edsr_gan_1024]

### 3.1.1 Imágenes de 256x256 a 1024x1024 (resolución x4)

Teniendo la siguiente imagen:

In [ ]:
input_256, target_256 = next(iter(low_res_valid_dataset))
plot_dataset(input=input_256.numpy(),target=target_256.numpy())

In [ ]:
for model,title in zip(model_list_256,names):
    plot_model_diff(model=model,input=input_256,target=target_256,figsize=(12,12),title=str(title+"_256"))


### 3.1.2 Imágenes de 512x512 a 1024x1024 (resolución x2)

Teniendo la siguiente imagen:

In [ ]:
input_512, target_512 = next(iter(med_res_valid_dataset))
plot_dataset(input=input_512.numpy(),target=target_512.numpy())

In [ ]:
for model,title in zip(model_list_512,names):
    plot_model_diff(model=model,input=input_512,target=target_512,figsize=(12,12),title=str(title+"_512"))

### 3.1.3 Imágenes de 1024x1024 a 1024x1024 (resolución x1)

Teniendo la siguiente imagen:

In [ ]:
input_1024, target_1024 = next(iter(high_res_valid_dataset))
plot_dataset(input=input_1024.numpy(),target=target_1024.numpy())

In [ ]:
vae_patch_reconstruct(vae_patch,input_1024,figsize=(8,8),title="PATCH_VAE")

for model,title in zip(model_list_1024,names):
    plot_model_diff(model=model,input=input_1024,target=target_1024,figsize=(12,12),title=str(title+"_1024"))

# 3.2 Tablas comparativas de métricas <a id="tcm"></a>

Función que, pasándole una imagen y un modelo, la divide en parches, predice cada parche y reconstruye la imagen:

In [ ]:
def patch_predict(model,single_image):
    patches = tf.image.extract_patches(
        images=single_image,  
        sizes=[1, 64, 64, 1], 
        strides=[1, 64, 64, 1], 
        rates=[1, 1, 1, 1], 
        padding='VALID'  
    )

    patches = tf.reshape(patches, [-1, 64, 64, 3])

    reconstructed_patches = model.predict(patches,verbose=0)

    reconstructed_image = np.zeros((1024, 1024, 3),dtype=np.float32)

    # Coloca cada parche en la imagen reconstruida
    patch_size = 64
    num_patches_per_row = 1024 // patch_size
    for i in range(len(reconstructed_patches)):
        row = i // num_patches_per_row
        col = i % num_patches_per_row
        reconstructed_image[row*patch_size:(row+1)*patch_size, col*patch_size:(col+1)*patch_size, :] = reconstructed_patches[i]
    return reconstructed_image

In [ ]:
data = {
        'Model': [],
        'PSNR_x1': [],
        'PSNR_x2': [],
        'PSNR_x4': [],
        'SSIM_x1' :[],
        'SSIM_x2' :[],
        'SSIM_x4' :[],
        'mae_x1' :[],
        'mae_x2' :[],
        'mae_x4' :[],
        'mse_x1' :[],
        'mse_x2' :[],
        'mse_x4' :[],
        }
df_one_image = pd.DataFrame(data)
df_mean = pd.DataFrame(data)

input_list = [input_256,input_512,input_1024]
target_list = [target_256,target_512,target_1024]

autoencoder_list = [autoencoder_256,autoencoder_512,autoencoder_1024]
gan_list = [gan_256,gan_512,gan_1024]
unet_gan_list = [unet_gan_256,unet_gan_512,unet_gan_1024]
edsr_gan_list = [edsr_gan_256,edsr_gan_512,edsr_gan_1024]


In [ ]:
def add_row(df,input_list,target_list,model_list,model_name,i):

    df.loc[i,'Model'] = model_name

    l = [4,2,1]
    if model_name == "VAE":
        l = [1]
        input_list = [input_list[2]]
        target_list = [target_list[2]]
    

    for j, input, target, model in zip(l,input_list, target_list, model_list):
        
        if model_name != "VAE":
            prediction = model.predict(input, verbose=0)[0]
        else:
            prediction = patch_predict(model,input)
            
        prediction = (prediction - np.min(prediction)) / (np.max(prediction) - np.min(prediction))

        # Calcular métricas
        psnr_val = tf.image.psnr(prediction, target, max_val=1)
        ssim_val = tf.image.ssim(prediction, target, max_val=1)
        mae_val = tf.keras.losses.mae(prediction, target)
        mse_val = tf.keras.losses.mse(prediction, target)

        #Asignar valores
        df.loc[i, f'PSNR_x{j}'] = psnr_val.numpy()
        df.loc[i, f'SSIM_x{j}'] = ssim_val.numpy()
        df.loc[i, f'mae_x{j}'] = np.mean(mae_val.numpy())
        df.loc[i, f'mse_x{j}'] = np.mean(mse_val.numpy())

In [ ]:
add_row(df_one_image,input_list,target_list,autoencoder_list,"Autoencoder",0)
add_row(df_one_image,input_list,target_list,[vae_patch],"VAE",1)
add_row(df_one_image,input_list,target_list,gan_list,"GAN",2)
add_row(df_one_image,input_list,target_list,unet_gan_list,"GAN UNET",3)
add_row(df_one_image,input_list,target_list,edsr_gan_list,"GAN EDSR",4)

In [ ]:
df_one_image

In [ ]:
df_one_image.to_excel("metrics_one_image.xlsx")

In [ ]:
def add_row_mean(df, dataset_list,model_list, model_name, i):

    df.loc[i, 'Model'] = model_name
    l = [4, 2, 1]

    if model_name == "VAE":
        l = [1]

    psnr_values = []
    ssim_values = []
    mae_values = []
    mse_values = []

    for j,dataset, model in zip(l,dataset_list, model_list):
        for input, target in dataset.as_numpy_iterator():
            if model_name != "VAE":
                prediction = model.predict(input, verbose=0)[0]
            else:
                prediction = patch_predict(model, input)

            prediction = (prediction - np.min(prediction)) / (np.max(prediction) - np.min(prediction))

            # Calcular métricas
            psnr_val = tf.image.psnr(prediction, target, max_val=1)
            ssim_val = tf.image.ssim(prediction, target, max_val=1)
            mae_val = tf.keras.losses.mae(prediction, target)
            mse_val = tf.keras.losses.mse(prediction, target)

            # Guardar valores
            psnr_values.append(psnr_val.numpy())
            ssim_values.append(ssim_val.numpy())
            mae_values.append(np.mean(mae_val.numpy()))
            mse_values.append(np.mean(mse_val.numpy()))

        df.loc[i, f'PSNR_x{j}'] = np.mean(psnr_values)
        df.loc[i, f'SSIM_x{j}'] = np.mean(ssim_values)
        df.loc[i, f'mae_x{j}'] = np.mean(mae_values)
        df.loc[i, f'mse_x{j}'] = np.mean(mse_values)

In [ ]:
dataset_list = [low_res_valid_dataset,med_res_valid_dataset,high_res_valid_dataset]

add_row_mean(df_mean,dataset_list,autoencoder_list,"Autoencoder",0)
add_row_mean(df_mean,[high_res_valid_dataset],[vae_patch],"VAE",1)
add_row_mean(df_mean,dataset_list,gan_list,"GAN",2)
add_row_mean(df_mean,dataset_list,unet_gan_list,"GAN UNET",3)
add_row_mean(df_mean,dataset_list,edsr_gan_list,"GAN EDSR",4)

In [ ]:
df_mean

In [ ]:
df_mean.to_excel("metrics_mean.xlsx")